
# FINN Preprocessor: Process MODIS Burned Area into polygon Shapefile

## 1. Setting Envoronments

### Systems settings

Most likely no need to be edited.

In [ ]:
# python libraries
import sys
import os
import re
import glob
import datetime
import subprocess
import shlex
from urllib.parse import urlparse
from importlib import reload
import gdal
import matplotlib.pylab as plt


# finn preproc codes
sys.path = sys.path + ['../code_anaconda']
import downloader
import af_import
import rst_import
import polygon_import
import run_step1
import run_step2
import export_shp
import plotter

In [ ]:
# database settings
os.environ['PGDATABASE'] = 'gis'
os.environ['PGUSER'] = 'finn'
os.environ['PGPASSWORD'] = 'finn'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'

Make sure that the PostGIS database is ready.

In [ ]:
# show info for the database
!psql postgres -c 'SELECT version();'
!pg_lsclusters

In [ ]:
# TODO i want to move this to Dockerfile somehow
# create plpython, needed only once for the database
try:
    p = subprocess.run(shlex.split("psql -d %s -c 'CREATE LANGUAGE plpython3u;'" % os.environ['PGDATABASE']), 
                       check=True, stderr=subprocess.PIPE)
except subprocess.CalledProcessError as e:
    if 'already exists' in e.stderr.decode():
        print(e.stderr.decode().replace('ERROR','OK').strip())
        pass

### Settings for Burnt Area Datasets 

MODIS burned area will be downloaded, if needed, for the region specified by the rectangle
1. `year_rst`, `month_rst`: MODIS burned area year/month to be processed
2. either  
  `four_corners`: (LowerLeft_lon, LowerLeft_Lat, UpperRight_lon, UpperRight_lat) or   
  `extent_shp`:  shape file (could be polygon of area of interest, points of fires)

In [ ]:
# MODIS raster datasets' year
#first_year_rst = 2017
#first_month_rst = 1
first_year_rst = 2018
first_month_rst = 1
last_year_rst = 2018
last_month_rst = 11

# tag to identify dataset
tag_bdt = 'modbdt_2018_tx'

In [ ]:
# Geographic extent of download
# specify either one of below (comment out one line with #)

four_corners = (-107, 25, -93, 37) # LL corner lon, LL corner LAT, UR corner Lon, UR corner Lat)
#extent_shp = './north_central_america.shp'  # shape file of North and Central America (i can create this from AllRegion polygon)

In [ ]:
# get year/month series
yrmo0 = datetime.date(first_year_rst, first_month_rst, 1)
yrmo1 = datetime.date(last_year_rst, last_month_rst, 1)
yrmos = [yrmo0 + datetime.timedelta(days=_) for _ in range((yrmo1-yrmo0).days)]
yrmos = [_ for _ in yrmos if _.day == 1]
yrmos.append(yrmo1)


# tags to identify datasets, automatically set to be modlct_YYYY, modvcf_YYYY
#tag_bdt = 'modbdt_%d%02d' % (year_rst, month_rst)
tags_bdt = ['modbdt_%d%02d' % (yrmo.year, yrmo.month) for yrmo in yrmos]
tags_bdt

---
## 2. Download raster datasets

Raster files URL and directories to save data

In [ ]:
# all downloads are stored in following dir
download_rootdir = '../downloads'

In [ ]:
# earthdata's URL for BA
#url_bdt = 'https://e4ftl01.cr.usgs.gov/MOTA/MCD64A1.006/%d.%02d.01/' % (year_rst, month_rst)
urls_bdt = ['https://e4ftl01.cr.usgs.gov/MOTA/MCD64A1.006/%d.%02d.01/' % (yrmo.year, yrmo.month) for yrmo in yrmos]

#ddir_bdt = download_rootdir +'/'+ ''.join(urlparse(url_bdt)[1:3])
ddirs_bdt = [download_rootdir +'/'+ ''.join(urlparse(url_bdt)[1:3]) for url_bdt in urls_bdt]

print('BDT downloads goes to %s' % ddirs_bdt)

Download burned area raster, <b>only for the tiles needed for the active fire file</b>

In [ ]:
if 'four_corners' in locals() and not four_corners is None:
    # use four corner
    poly = "POLYGON((%f %f, %f %f, %f %f, %f %f, %f %f))" % (
        four_corners[0], four_corners[1], 
        four_corners[0], four_corners[3], 
        four_corners[2], four_corners[3], 
        four_corners[2], four_corners[1],
        four_corners[0], four_corners[1], 
    )
    knd = 'wkt'
elif 'extent_shp' in locals() and not extent_shp is None:
    # use shape file
    poly = extent_shp
    knd = 'ds'
else:
    raise RuntimeError('Specify region of interest!')
        

In [ ]:
reload(downloader)
for url_bdt in urls_bdt:
    downloader.download_only_needed(url = url_bdt, droot = download_rootdir, region=poly, region_knd=knd)

Verify BDT files' checksum.  If a file is corrupted, the file is downloaded again.

In [ ]:
for ddir_bdt, url_bdt in zip(ddirs_bdt, urls_bdt):
    downloader.purge_corrupted(ddir = ddir_bdt, url=url_bdt)

## 3. Import raster datasets

Downloaded files need preprocessing, which is to extract the only raster band needed, and also make coordinate system to be WGS84.  Intermediate files are created in following directories.

In [ ]:
workdir_bdt = '../proc_rst_%s' % tag_bdt

print('BDT preprocessing occurs in %s' % workdir_bdt)

### Import Buned Area

First grab hdf file names from the download directory

In [ ]:
fnames_bdt = {}
n = 0
for ddir_bdt, yrmo in zip(ddirs_bdt, yrmos):
    search_string = "%(ddir_bdt)s/MCD64A1.A%(year_rst)s???.h??v??.006.*.hdf" % dict(
            ddir_bdt = ddir_bdt, year_rst=yrmo.year)
    fnames_bdt[yrmo] = sorted(glob.glob(search_string))
    n += len(fnames_bdt[yrmo])
print('found %d hdf files' % n )
if n == 0:
    raise RuntimeError("check if downloads are successful and search string to be correct: %s" % search_string)

The next command performs three tasks, "merge", "resample" and "import".  First two task creates intermediate GeoTiff files in <i>work_dir</i>.  Last task actually import the data into database's <i>raster</i> schema.

You can run only selected tasks with run_XXX flags to `False`, when you know that processing failed in the middle and you resolved the issue.

In [ ]:
reload(rst_import)
for yrmo in yrmos:
    print(yrmo)
    rst_import.main(tag_bdt, fnames=fnames_bdt[yrmo], workdir = workdir_bdt, 
                    run_merge=True, run_resample=True, run_import=True)

In [ ]:
# merge shapes into one
import ogr
for knd in ('poly', 'pnt'):
    shpfiles = glob.glob(os.path.join(workdir_bdt, 'rsp', 'MCD64A1.A???????.%s.shp' % knd))
    oname = os.path.join(workdir_bdt, '.'.join(('MCD64A1', tag_bdt, knd, 'shp' )))
    if os.path.exists(oname):
        drv = ogr.GetDriverByName('ESRI Shapefile')
        drv.DeleteDataSource(oname)
    cmd = ['ogr2ogr', '-update', '-append', oname, ]
    for shp in shpfiles:
        cmdx = cmd + [shp]
        subprocess.run(cmdx, check=True)